In [1]:
import os

import pandas as pd 
import numpy as np

<h3>전처리할 데이터 읽기</h3>

In [2]:
# 원본 교통량 데이터 읽기
xls = pd.read_excel('./data/traffic_process/수도권 교통량.xls', converters={'집계일자': pd.to_datetime}, sheet_name=None)

In [3]:
# 모든 시트 데이터를 한개에 데이터 프레임으로 합치기
df_traffic = pd.concat(list(xls.values()), ignore_index=True)
df_traffic.head(10)

,집계일자,집계시,영업소코드,영업소명,입출구구분코드,입출구구분명,TCS하이패스구분코드,TCS하이패스구분명,N1종교통량,N2종교통량,N3종교통량,N4종교통량,N5종교통량,N6종교통량,총교통량
0,2021-07-01,0,254,군자,0,입구,1,TCS,68,1,4,10,3,1,87
1,2021-07-01,0,254,군자,0,입구,2,HIPASS,278,25,11,31,28,18,391
2,2021-07-01,0,254,군자,1,출구,1,TCS,59,3,2,10,4,5,83
3,2021-07-01,0,254,군자,1,출구,2,HIPASS,302,22,8,43,42,22,439
4,2021-07-01,1,254,군자,0,입구,1,TCS,33,0,1,8,1,0,43
5,2021-07-01,1,254,군자,0,입구,2,HIPASS,175,27,15,20,33,8,278
6,2021-07-01,1,254,군자,1,출구,1,TCS,45,2,1,16,7,8,79
7,2021-07-01,1,254,군자,1,출구,2,HIPASS,206,36,14,34,38,17,345
8,2021-07-01,2,254,군자,0,입구,1,TCS,17,1,4,10,4,0,36
9,2021-07-01,2,254,군자,0,입구,2,HIPASS,131,27,9,25,42,6,240


In [4]:
df_traffic.tail(10)

,집계일자,집계시,영업소코드,영업소명,입출구구분코드,입출구구분명,TCS하이패스구분코드,TCS하이패스구분명,N1종교통량,N2종교통량,N3종교통량,N4종교통량,N5종교통량,N6종교통량,총교통량
228086,2022-07-31,21,101,서울,1,출구,1,TCS,458,4,10,5,0,43,520
228087,2022-07-31,21,101,서울,1,출구,2,HIPASS,4115,46,257,65,20,157,4660
228088,2022-07-31,22,101,서울,0,입구,1,TCS,364,2,1,10,0,1,378
228089,2022-07-31,22,101,서울,0,입구,2,HIPASS,3212,65,207,55,21,119,3679
228090,2022-07-31,22,101,서울,1,출구,1,TCS,422,9,4,8,0,39,482
228091,2022-07-31,22,101,서울,1,출구,2,HIPASS,3634,58,253,63,19,113,4140
228092,2022-07-31,23,101,서울,0,입구,1,TCS,247,1,2,5,1,3,259
228093,2022-07-31,23,101,서울,0,입구,2,HIPASS,2102,67,164,57,19,116,2525
228094,2022-07-31,23,101,서울,1,출구,1,TCS,419,9,5,7,0,37,477
228095,2022-07-31,23,101,서울,1,출구,2,HIPASS,3402,81,227,66,16,96,3888


In [5]:
df_traffic.describe()

,집계시,영업소코드,입출구구분코드,TCS하이패스구분코드,N1종교통량,N2종교통량,N3종교통량,N4종교통량,N5종교통량,N6종교통량,총교통량
count,228096.000000,228096.000000,228096.000000,228096.000000,228096.000000,228096.000000,228096.000000,228096.000000,228096.000000,228096.000000,228096.000000
mean,11.500000,355.666667,0.500000,1.500000,1142.880007,37.102189,54.014424,36.895671,40.668179,51.849791,1363.410261
std,6.922202,226.884952,0.500001,0.500001,1423.372395,51.389522,92.320498,36.655116,66.701286,62.900062,1626.489354
min,0.000000,101.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.000000
25%,5.750000,190.000000,0.000000,1.000000,169.000000,4.000000,8.000000,10.000000,3.000000,5.000000,245.000000
50%,11.500000,253.500000,0.500000,1.500000,402.000000,14.000000,26.000000,24.000000,11.000000,25.000000,513.000000
75%,17.250000,651.000000,1.000000,2.000000,1786.000000,48.000000,56.000000,52.000000,48.000000,81.000000,2116.000000
max,23.000000,685.000000,1.000000,2.000000,13012.000000,813.000000,944.000000,652.000000,1624.000000,769.000000,17300.000000


<h3>영업소별 총교통량 합 계산</h3>

In [6]:
# 총교통량을 [n(날짜*영업소개수*24시간), 4(입/출구)] 형태로 변경
traffic_matrix = df_traffic['총교통량'].values.reshape(-1, 4)
print(traffic_matrix.shape)
traffic_matrix

(57024, 4)


array([[  87,  391,   83,  439],
       [  43,  278,   79,  345],
       [  36,  240,   56,  279],
       ...,
       [ 419, 4322,  520, 4660],
       [ 378, 3679,  482, 4140],
       [ 259, 2525,  477, 3888]])

In [7]:
# 영업소별 입/출구 교통량 합 계산
traffic_sum = traffic_matrix.sum(1)
print(traffic_sum.shape)
traffic_sum

(57024,)


array([1000,  745,  611, ..., 9921, 8679, 7149])

In [8]:
# 총교통량을 [n(날짜), 6(영업소개수), 24(시간)] 형태로 변경
traffic_sum = traffic_sum.reshape(-1, 6, 24)
print(traffic_sum.shape)

(396, 6, 24)


In [9]:
# 모든 영업소별 교통량 합 계산
traffic_sum = traffic_sum.sum(1)
print(traffic_sum.shape)

(396, 24)


<h3> 기존 데이터에서 집계일자&집계시 중복 제거 및 추출 </h3>

In [10]:
# 4개 데이터 기준으로 데이터 추출
df_traffic_sum = df_traffic.iloc[::4] #, [0, 1]
df_traffic_sum

,집계일자,집계시,영업소코드,영업소명,입출구구분코드,입출구구분명,TCS하이패스구분코드,TCS하이패스구분명,N1종교통량,N2종교통량,N3종교통량,N4종교통량,N5종교통량,N6종교통량,총교통량
0,2021-07-01,0,254,군자,0,입구,1,TCS,68,1,4,10,3,1,87
4,2021-07-01,1,254,군자,0,입구,1,TCS,33,0,1,8,1,0,43
8,2021-07-01,2,254,군자,0,입구,1,TCS,17,1,4,10,4,0,36
12,2021-07-01,3,254,군자,0,입구,1,TCS,34,3,6,15,7,0,65
16,2021-07-01,4,254,군자,0,입구,1,TCS,56,5,7,14,11,1,94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228076,2022-07-31,19,101,서울,0,입구,1,TCS,395,4,8,2,3,3,415
228080,2022-07-31,20,101,서울,0,입구,1,TCS,366,5,8,4,0,1,384
228084,2022-07-31,21,101,서울,0,입구,1,TCS,397,4,3,7,3,5,419
228088,2022-07-31,22,101,서울,0,입구,1,TCS,364,2,1,10,0,1,378


In [11]:
# 영업소명으로 멀티 인덱스 생성
df_traffic_multi = df_traffic_sum.set_index(['집계일자', '영업소명', '집계시'])
df_traffic_multi

영업소코드  입출구구분코드 입출구구분명  TCS하이패스구분코드 TCS하이패스구분명  N1종교통량  \
집계일자       영업소명 집계시                                                          
2021-07-01 군자   0      254        0     입구            1        TCS      68   
                1      254        0     입구            1        TCS      33   
                2      254        0     입구            1        TCS      17   
                3      254        0     입구            1        TCS      34   
                4      254        0     입구            1        TCS      56   
...                    ...      ...    ...          ...        ...     ...   
2022-07-31 서울   19     101        0     입구            1        TCS     395   
                20     101        0     입구            1        TCS     366   
                21     101        0     입구            1        TCS     397   
                22     101        0     입구            1        TCS     364   
                23     101        0     입구            1        TCS     247   

                     N2종교통량  N3종교통량  N4종교통량  N5종교통량  N6종교통량  총교통량  
집계일자       영업소명 집계시                                                
2021-07-01 군자   0         1       4      10       3       1    87  
                1         0       1       8       1       0    43  
                2         1       4      10       4       0    36  
                3         3       6      15       7       0    65  
                4         5       7      14      11       1    94  
...                     ...     ...     ...     ...     ...   ...  
2022-07-31 서울   19        4       8       2       3       3   415  
                20        5       8       4       0       1   384  
                21        4       3       7       3       5   419  
                22        2       1      10       0       1   378  
                23        1       2       5       1       3   259  

[57024 rows x 12 columns]

In [12]:
# 멀티 인덱스를 사용하여 '군자' 영업소를 제외한 모든 영업소 제거
idx = pd.IndexSlice
df_traffic_multi_select = df_traffic_multi.loc[idx[:, '군자' ,:], :]
df_traffic_multi_select

영업소코드  입출구구분코드 입출구구분명  TCS하이패스구분코드 TCS하이패스구분명  N1종교통량  \
집계일자       영업소명 집계시                                                          
2021-07-01 군자   0      254        0     입구            1        TCS      68   
                1      254        0     입구            1        TCS      33   
                2      254        0     입구            1        TCS      17   
                3      254        0     입구            1        TCS      34   
                4      254        0     입구            1        TCS      56   
...                    ...      ...    ...          ...        ...     ...   
2022-07-31 군자   19     254        0     입구            1        TCS     184   
                20     254        0     입구            1        TCS     136   
                21     254        0     입구            1        TCS     159   
                22     254        0     입구            1        TCS     105   
                23     254        0     입구            1        TCS      84   

                     N2종교통량  N3종교통량  N4종교통량  N5종교통량  N6종교통량  총교통량  
집계일자       영업소명 집계시                                                
2021-07-01 군자   0         1       4      10       3       1    87  
                1         0       1       8       1       0    43  
                2         1       4      10       4       0    36  
                3         3       6      15       7       0    65  
                4         5       7      14      11       1    94  
...                     ...     ...     ...     ...     ...   ...  
2022-07-31 군자   19        2       2       5       1       2   196  
                20        2       2       3       1       1   145  
                21        4       2       6       1       0   172  
                22        1       1       4       1       0   112  
                23        0       2       6       3       0    95  

[9504 rows x 12 columns]

In [13]:
# 정리된 집계일자 와 집계시만 추출
df_index = df_traffic_multi_select.reset_index().iloc[:, [0, 2]]
df_index

,집계일자,집계시
0,2021-07-01,0
1,2021-07-01,1
2,2021-07-01,2
3,2021-07-01,3
4,2021-07-01,4
...,...,...
9499,2022-07-31,19
9500,2022-07-31,20
9501,2022-07-31,21
9502,2022-07-31,22


<h3>계산된 전체 총 교통량과 인덱스 시간 인덱스 합치기</h3>

In [14]:
df_index['총교통량'] = traffic_sum.flatten()
df_index

,집계일자,집계시,총교통량
0,2021-07-01,0,9030
1,2021-07-01,1,6469
2,2021-07-01,2,5238
3,2021-07-01,3,5834
4,2021-07-01,4,11062
...,...,...,...
9499,2022-07-31,19,36775
9500,2022-07-31,20,35230
9501,2022-07-31,21,34829
9502,2022-07-31,22,31239


<h3>마지막 7월달 데이터 제거</h3>

In [21]:
# 1년치 학습 데이터 추출
data_size_of_7 = 31*24
df_train = df_index.iloc[:-data_size_of_7, :]
df_train

,집계일자,집계시,총교통량
0,2021-07-01,0,9030
1,2021-07-01,1,6469
2,2021-07-01,2,5238
3,2021-07-01,3,5834
4,2021-07-01,4,11062
...,...,...,...
8755,2022-06-30,19,35395
8756,2022-06-30,20,27448
8757,2022-06-30,21,24237
8758,2022-06-30,22,18403


<h3>날짜 데이터를 숫자형으로 변경</h3>

In [35]:
df_train_date = df_train.copy()

In [36]:
df_train_date['집계일자'] = df_train['집계일자'].dt.dayofyear
df_train_date.columns = ['날짜', '시간', '총교통량']
df_train_date = df_train_date.set_index(['날짜', '시간'])
df_train_date

총교통량
날짜  시간       
182 0    9030
    1    6469
    2    5238
    3    5834
    4   11062
...       ...
181 19  35395
    20  27448
    21  24237
    22  18403
    23  13044

[8760 rows x 1 columns]

<h3>날씨 데이터 합치기</h3>

In [37]:
# 날씨 데이터 읽기
df_weather = pd.read_csv("./data/cycle_sum/train_preprocess.csv")
df_weather = df_weather.iloc[:, [0, 1, 4, 5]]
df_weather = df_weather.set_index(['날짜', '시간'])
df_weather

기온  강수량
날짜  시간           
182 0   24.0  0.0
    1   24.0  0.0
    2   24.0  0.0
    3   23.0  0.0
    4   23.0  0.0
...      ...  ...
181 19  20.0  3.3
    20  21.0  2.5
    21  21.0  2.0
    22  21.0  1.0
    23  22.0  0.0

[8760 rows x 2 columns]

In [42]:
# 데이터 합치기
df_result = pd.concat([df_train_date, df_weather], axis=1)
df_result

총교통량    기온  강수량
날짜  시간                  
182 0    9030  24.0  0.0
    1    6469  24.0  0.0
    2    5238  24.0  0.0
    3    5834  23.0  0.0
    4   11062  23.0  0.0
...       ...   ...  ...
181 19  35395  20.0  3.3
    20  27448  21.0  2.5
    21  24237  21.0  2.0
    22  18403  21.0  1.0
    23  13044  22.0  0.0

[8760 rows x 3 columns]

<h3>전처리 완료된 데이터 저장</h3>

In [ ]:
df_result.to_csv('./data/traffic_process/train.csv')